<a href="https://colab.research.google.com/github/AshbeeKim/Kaggle_task/blob/main/NYC_taxi_trip_duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os  
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/data/kaggle"   
from google.colab import drive  
drive.mount('/content/drive')  
!cd /content/drive/MyDrive/data/kaggle  
!kaggle competitions download -c nyc-taxi-trip-duration
!unzip train.zip
!unzip test.zip
!unzip sample_submission.zip
!ls  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 25% 5.00M/20.3M [00:00<00:00, 37.2MB/s]
100% 20.3M/20.3M [00:00<00:00, 80.7MB/s]
 79% 50.0M/62.9M [00:00<00:00, 50.1MB/s]
100% 62.9M/62.9M [00:00<00:00, 86.1MB/s]
  0% 0.00/2.49M [00:00<?, ?B/s]
100% 2.49M/2.49M [00:00<00:00, 168MB/s]
Archive:  train.zip
  inflating: train.csv               
Archive:  test.zip
  inflating: test.csv                
Archive:  sample_submission.zip
  inflating: sample_submission.csv   
drive	     sample_submission.csv  test.csv  train.csv
sample_data  sample_submission.zip  test.zip  train.zip


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn
# import mglearn
import graphviz
import warnings
warnings.filterwarnings('ignore')
# import statsmodels.api as sm

# 최대 줄 수 설정
pd.set_option('display.max_rows', 1500)
# 최대 열 수 설정
pd.set_option('display.max_columns', 1500)
# 표시할 가로의 길이
pd.set_option('display.width', 1500)

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
samplesubmisssion = pd.read_csv('./sample_submission.csv')

In [ ]:
data = train.copy()

In [ ]:
print(data.shape)
print(test.shape)
print(samplesubmisssion.shape)
print('='*150)
print(data.info())
print(test.info())
print(samplesubmisssion.info())
print('='*150)
print(data.describe().round(3))
print('-'*150)
print(test.describe().round(3))
print('-'*150)
print(samplesubmisssion.describe().round(3))

(1458644, 11)
(625134, 9)
(625134, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625134 entries, 0 to 625133
Data columns (total 9 columns):
 #   Co

In [ ]:
print(data.columns) #'dropoff_datetime', 'trip_duration'는 test에 없고 train에만 있는 데이터, samplesubmission에는 'id'와 'trip_duration'만 있음
print('-'*250)
print(test.columns)
print('-'*250)
print(samplesubmisssion.columns)

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration'], dtype='object')
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Index(['id', 'vendor_id', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag'], dtype='object')
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Index(['id', 'trip_duration'], dtype='object')


In [ ]:
print(data.head())

          id  vendor_id      pickup_datetime     dropoff_datetime  passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude store_and_fwd_flag  trip_duration
0  id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30                1        -73.982155        40.767937         -73.964630         40.765602                  N            455
1  id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38                1        -73.980415        40.738564         -73.999481         40.731152                  N            663
2  id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48                1        -73.979027        40.763939         -74.005333         40.710087                  N           2124
3  id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40                1        -74.010040        40.719971         -74.012268         40.706718                  N            429
4  id2181028          2  2016-03-26 13:30:55  2016-03-2

In [ ]:
print(data.tail())

                id  vendor_id      pickup_datetime     dropoff_datetime  passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude store_and_fwd_flag  trip_duration
1458639  id2376096          2  2016-04-08 13:31:04  2016-04-08 13:44:02                4        -73.982201        40.745522         -73.994911         40.740170                  N            778
1458640  id1049543          1  2016-01-10 07:35:15  2016-01-10 07:46:10                1        -74.000946        40.747379         -73.970184         40.796547                  N            655
1458641  id2304944          2  2016-04-22 06:57:41  2016-04-22 07:10:25                1        -73.959129        40.768799         -74.004433         40.707371                  N            764
1458642  id2714485          1  2016-01-05 15:56:26  2016-01-05 16:02:39                1        -73.982079        40.749062         -73.974632         40.757107                  N            373
1458643  id1209952       

In [ ]:
print(test.head())

          id  vendor_id      pickup_datetime  passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude store_and_fwd_flag
0  id3004672          1  2016-06-30 23:59:58                1        -73.988129        40.732029         -73.990173         40.756680                  N
1  id3505355          1  2016-06-30 23:59:53                1        -73.964203        40.679993         -73.959808         40.655403                  N
2  id1217141          1  2016-06-30 23:59:47                1        -73.997437        40.737583         -73.986160         40.729523                  N
3  id2150126          2  2016-06-30 23:59:41                1        -73.956070        40.771900         -73.986427         40.730469                  N
4  id1598245          1  2016-06-30 23:59:33                1        -73.970215        40.761475         -73.961510         40.755890                  N


In [ ]:
print(test.tail())

               id  vendor_id      pickup_datetime  passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude store_and_fwd_flag
625129  id3008929          1  2016-01-01 00:02:52                1        -74.003464        40.725105         -74.001251         40.733643                  N
625130  id3700764          1  2016-01-01 00:01:52                1        -74.006363        40.743782         -73.953407         40.782467                  N
625131  id2568735          1  2016-01-01 00:01:24                2        -73.972267        40.759865         -73.876602         40.748665                  N
625132  id1384355          1  2016-01-01 00:00:28                1        -73.976501        40.733562         -73.854263         40.891788                  N
625133  id0621643          2  2016-01-01 00:00:22                2        -73.981850        40.716881         -73.969330         40.769379                  N


In [ ]:
print(samplesubmisssion.head())

          id  trip_duration
0  id3004672            959
1  id3505355            959
2  id1217141            959
3  id2150126            959
4  id1598245            959


In [ ]:
print(samplesubmisssion.tail())

               id  trip_duration
625129  id3008929            959
625130  id3700764            959
625131  id2568735            959
625132  id1384355            959
625133  id0621643            959


'dropoff_datetime', 'trip_duration'는 test에 없고 train에만 있는 데이터, samplesubmission에는 'id'와 'trip_duration'만 있음

y에 dropoff_datetime과 trip_duration으로 맞춰야 하는가...?

# Kaggle
The competition dataset is based on the 2016 NYC Yellow Cab trip record data made available in Big Query on Google Cloud Platform. The data was originally published by the NYC Taxi and Limousine Commission (TLC). The data was sampled and cleaned for the purposes of this playground competition. Based on individual trip attributes, participants should predict the duration of each trip in the test set.

## File descriptions
*  train.csv - the training set (contains 1458644 trip records)
*  test.csv - the testing set (contains 625134 trip records)
*  sample_submission.csv - a sample submission file in the correct format

## Data fields
*  id - a unique identifier for each trip
*  vendor_id - a code indicating the provider associated with the trip record
*  pickup_datetime - date and time when the meter was engaged
*  dropoff_datetime - date and time when the meter was disengaged
*  passenger_count - the number of passengers in the vehicle (driver entered value)
*  pickup_longitude - the longitude where the meter was engaged
*  pickup_latitude - the latitude where the meter was engaged
*  dropoff_longitude - the longitude where the meter was disengaged
*  dropoff_latitude - the latitude where the meter was disengaged
*  store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a *  connection to the server - Y=store and forward; N=not a store and forward trip
*  trip_duration - duration of the trip in seconds
Disclaimer: The decision was made to not remove dropoff coordinates from the dataset order to provide an expanded set of variables to use in Kernels.

In [ ]:
print(len(data['id'].unique()))
print(len(test['id'].unique()))

print(len(data['vendor_id'].unique()))
print(len(test['vendor_id'].unique()))

print(len(data['passenger_count'].unique()))
print(len(test['passenger_count'].unique()))

TypeError: ignored

In [ ]:
x=0
for i in data['passenger_count'].unique():
  x+=1
print(x)
x=0
for i in test['vendor_id'].unique():
  x+=1
print(x)

2
2


In [ ]:
print(len(data['id'].unique()))
print(len(test['id'].unique()))

1458644
625134
